In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-01 17:15:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  23.5MB/s    in 3m 23s  

2022-05-01 17:18:56 (31.7 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-01 17:19:27--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [7]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB2(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

37445632/37432240 [==============================] - 0s 0us/step


In [9]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(260, 260))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

1%, Top 1:0.76 Top 5:0.956
2%, Top 1:0.763 Top 5:0.942
3%, Top 1:0.7766666666666666 Top 5:0.9386666666666666
4%, Top 1:0.774 Top 5:0.9335
5%, Top 1:0.774 Top 5:0.9356
6%, Top 1:0.7703333333333333 Top 5:0.936
7%, Top 1:0.7671428571428571 Top 5:0.9388571428571428
8%, Top 1:0.76875 Top 5:0.93675
9%, Top 1:0.7702222222222223 Top 5:0.9366666666666666
10%, Top 1:0.7694 Top 5:0.9356
11%, Top 1:0.7703636363636364 Top 5:0.9356363636363636
12%, Top 1:0.7698333333333334 Top 5:0.9361666666666667
13%, Top 1:0.7729230769230769 Top 5:0.9361538461538461
14%, Top 1:0.7725714285714286 Top 5:0.9364285714285714
15%, Top 1:0.7732 Top 5:0.9362666666666667
16%, Top 1:0.7725 Top 5:0.936125
17%, Top 1:0.7725882352941177 Top 5:0.936
18%, Top 1:0.7715555555555556 Top 5:0.9356666666666666
19%, Top 1:0.7717894736842106 Top 5:0.9351578947368421
20%, Top 1:0.771 Top 5:0.9345
21%, Top 1:0.7705714285714286 Top 5:0.9347619047619048
22%, Top 1:0.7696363636363637 Top 5:0.9342727272727273
23%, Top 1:0.7688695652173914 Top